In [1]:
import pandas as pd
import numpy as np
import re
from string import digits
import numpy as np
import pandas as pd
import string, sys
from sklearn import metrics as evaluators
from sklearn.model_selection import cross_validate
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import BorderlineSMOTE
from collections import Counter

In [2]:
import pandas as pd
import numpy as np
import re
from string import digits
import numpy as np
import pandas as pd
import string, sys
from sklearn import metrics as evaluators
from sklearn.model_selection import cross_validate
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import BorderlineSMOTE
from collections import Counter

In [3]:
server = SSHTunnelForwarder(('140.127.32.24',22),
    ssh_password='npustMIS2023',
    ssh_username='chiayu',
    remote_bind_address=('140.127.32.23', 27016)
)

mongo_user = 'senior01'
mongo_password = 'senior018787'

server.start()
# print(server.local_bind_port)
client = MongoClient('127.0.0.1',server.local_bind_port)
client.admin.authenticate(mongo_user,mongo_password)
#db_name = client['medicine']
print('Done')

Done


In [4]:
db_name = client['medicine']
data = db_name['IM_alldata'].find()
agedata = db_name['HELNO_CHTNO'].find()
datadf = pd.DataFrame(list(data))
agedata = pd.DataFrame(list(agedata))
print(datadf)
print(agedata)

                            _id    年份            健診號       病歷號  性別   年齡   尿酸  \
0      6564a85c16d962e4e681866a  2010  2010010401001   9764229   1  0.0  4.6   
1      6564a85c16d962e4e681866b  2010  2010010401003  20834735   1  2.0  6.5   
2      6564a85c16d962e4e681866c  2010  2010010401005   2834286   0  2.0  3.1   
3      6564a85c16d962e4e681866d  2010  2010010401006  20863610   1  1.0  5.5   
4      6564a85c16d962e4e681866e  2010  2010010401009   3901976   0  1.0  4.7   
...                         ...   ...            ...       ...  ..  ...  ...   
59138  6564b73a16d962e4e6826d6c  2014  2014123101068    539434   1  NaN  9.3   
59139  6564b73a16d962e4e6826d6d  2014  2014123101069   8000050   1  1.0  5.2   
59140  6564b73a16d962e4e6826d6e  2014  2014123101071   2147418   0  1.0  4.3   
59141  6564b73a16d962e4e6826d6f  2014  2014123101073  39118655   1  1.0  5.1   
59142  6564b73a16d962e4e6826d70  2014  2014123101076  39033805   1  NaN  NaN   

       紅血球色素    血小板   淋巴球  ...   齲齒  牙齒

In [5]:
HELNO = []
age_1 = []
for i in range(0,len(agedata)):
    HEL = agedata['健診號'][i]
    now = int(HEL[0:4]) - int(agedata['出生年'][i])
    HELNO.append(agedata['健診號'][i])
    age_1.append(now)
print(len(HELNO))
print(len(age_1))

70993
70993


In [6]:
for i in range(0,len(datadf)):
    try:
        temp = HELNO.index(datadf['健診號'][i])       
        datadf['年齡'][i] = int(age_1[temp])
    except:
        pass
print(datadf)

                            _id    年份            健診號       病歷號  性別    年齡   尿酸  \
0      6564a85c16d962e4e681866a  2010  2010010401001   9764229   1  33.0  4.6   
1      6564a85c16d962e4e681866b  2010  2010010401003  20834735   1  74.0  6.5   
2      6564a85c16d962e4e681866c  2010  2010010401005   2834286   0  73.0  3.1   
3      6564a85c16d962e4e681866d  2010  2010010401006  20863610   1  46.0  5.5   
4      6564a85c16d962e4e681866e  2010  2010010401009   3901976   0  51.0  4.7   
...                         ...   ...            ...       ...  ..   ...  ...   
59138  6564b73a16d962e4e6826d6c  2014  2014123101068    539434   1  42.0  9.3   
59139  6564b73a16d962e4e6826d6d  2014  2014123101069   8000050   1  54.0  5.2   
59140  6564b73a16d962e4e6826d6e  2014  2014123101071   2147418   0  46.0  4.3   
59141  6564b73a16d962e4e6826d6f  2014  2014123101073  39118655   1  41.0  5.1   
59142  6564b73a16d962e4e6826d70  2014  2014123101076  39033805   1  39.0  NaN   

       紅血球色素    血小板   淋巴球  

In [7]:
maxage = 0
minage = 100
for i in range(0,len(datadf)):
    if datadf['年齡'][i] > maxage:
        maxage = datadf['年齡'][i]
    if datadf['年齡'][i] < minage:
        minage = datadf['年齡'][i]
print(minage)
print(maxage)

2.0
101.0


In [8]:
datadf['年齡分群'] = 0
for i in range(0,len(datadf)):
    if datadf['年齡'][i] < 40:
        datadf['年齡分群'][i] = 0
    elif datadf['年齡'][i] >= 40 and datadf['年齡'][i] < 70:
        datadf['年齡分群'][i] = 1
    elif datadf['年齡'][i] >= 70:
        datadf['年齡分群'][i] = 2
print(pd.Series.value_counts(datadf['年齡分群']))

年齡分群
1    41201
0    13297
2     4645
Name: count, dtype: int64


In [9]:
for i in range(0,len(datadf)):
    body = datadf['BMI'][i]
    if body < 18.5:
        datadf['BMI'][i] = 1
    elif body >= 18.5 and body < 24:
        datadf['BMI'][i] = 2
    elif body >= 24 and body < 27:
        datadf['BMI'][i] = 3
    elif body >= 27:
        datadf['BMI'][i] = 4
print(pd.Series.value_counts(datadf['BMI']))

BMI
2.0    28428
3.0    16800
4.0    10625
1.0     2694
Name: count, dtype: int64


In [10]:
column = ['健診號', '病歷號', '性別', '年齡', '尿酸', '紅血球色素', '血小板', '淋巴球',
       '嗜中性球', '單核球', '飯前血糖', '飯後血糖', '膽固醇總量', 'BMI', '腰圍', '三酸甘油脂', 'HDL',
       '收縮壓', '舒張壓', '低密度脂蛋白', '超低密度脂蛋白', 'Atherogenic Index of Plasma',
       'Triglyceride Glucose Index', 'Monocyte to Lymphocyte Ratio', 'SIRI',
       'SII', '大腸息肉', 'HP', '胃息肉', '切片腺瘤', '增生性息肉', '腺性息肉', '大腸切片息肉', '腸黏膜化生',
       '牙結石', '齲齒', '牙齒缺失', '慢性齒齦炎', '胃潰瘍', '胃炎', '膽囊息肉', '膽囊結石', '脂肪肝', 'AST',
       'Hs-CRP', '年齡分群']

boy0 = pd.DataFrame(columns = column)
boy1 = pd.DataFrame(columns = column)
boy2 = pd.DataFrame(columns = column)
girl0 = pd.DataFrame(columns = column)
girl1 = pd.DataFrame(columns = column)
girl2 = pd.DataFrame(columns = column)
with tqdm(total=len(datadf)) as pbar:
    for i in range(0,len(datadf)):
        tempdf = datadf.iloc[[i]]  # 使用雙重中括號將其轉換為 DataFrame
        #tempdf.reset_index(drop=True, inplace=True)  # 重新設置索引
        if datadf['性別'][i] == 0:
            if datadf['年齡分群'][i] == 0:
                #tempdf = datadf.iloc[i]
                girl0 = pd.concat([girl0, tempdf], ignore_index=True)
            elif datadf['年齡分群'][i] == 1:
                #tempdf = datadf.iloc[i]
                girl1 = pd.concat([girl1, tempdf], ignore_index=True)
            elif datadf['年齡分群'][i] == 2:
                #tempdf = datadf.iloc[i]
                girl2 = pd.concat([girl2, tempdf], ignore_index=True)
        elif datadf['性別'][i] == 1:
            if datadf['年齡分群'][i] == 0:
                #tempdf = datadf.iloc[i]
                boy0 = pd.concat([boy0, tempdf], ignore_index=True)
            elif datadf['年齡分群'][i] == 1:
                #tempdf = datadf.iloc[i]
                boy1 = pd.concat([boy1, tempdf], ignore_index=True)
            elif datadf['年齡分群'][i] == 2:
                #tempdf = datadf.iloc[i]
                boy2 = pd.concat([boy2, tempdf], ignore_index=True)
        pbar.update(1)
print(pd.Series.value_counts(boy0['年齡分群']))
print(pd.Series.value_counts(boy1['年齡分群']))
print(pd.Series.value_counts(boy2['年齡分群']))
print(pd.Series.value_counts(girl0['年齡分群']))
print(pd.Series.value_counts(girl1['年齡分群']))
print(pd.Series.value_counts(girl2['年齡分群']))

100%|███████████████████████████████████████████████████████████████████████████| 59143/59143 [00:47<00:00, 1238.38it/s]

年齡分群
0    8175
Name: count, dtype: int64
年齡分群
1    25310
Name: count, dtype: int64
年齡分群
2    2433
Name: count, dtype: int64
年齡分群
0    5122
Name: count, dtype: int64
年齡分群
1    15891
Name: count, dtype: int64
年齡分群
2    2212
Name: count, dtype: int64


In [11]:
datadf['性別分群'] = 0
for i in range(0,len(datadf)):
    if datadf['性別'][i] == 0:
        datadf['性別分群'][i] = 0
    elif datadf['性別'][i] == 1:
        datadf['性別分群'][i] = 1
print(pd.Series.value_counts(datadf['性別分群']))

性別分群
1    35918
0    23225
Name: count, dtype: int64


In [ ]:
column = ['健診號', '病歷號', '性別', '年齡', '尿酸', '紅血球色素', '血小板', '淋巴球',
       '嗜中性球', '單核球', '飯前血糖', '飯後血糖', '膽固醇總量', 'BMI', '腰圍', '三酸甘油脂', 'HDL',
       '收縮壓', '舒張壓', '低密度脂蛋白', '超低密度脂蛋白', 'Atherogenic Index of Plasma',
       'Triglyceride Glucose Index', 'Monocyte to Lymphocyte Ratio', 'SIRI',
       'SII', '大腸息肉', 'HP', '胃息肉', '切片腺瘤', '增生性息肉', '腺性息肉', '大腸切片息肉', '腸黏膜化生',
       '牙結石', '齲齒', '牙齒缺失', '慢性齒齦炎', '胃潰瘍', '胃炎', '膽囊息肉', '膽囊結石', '脂肪肝', 'AST',
       'Hs-CRP', '性別分群']
boy = pd.DataFrame(columns = column)
girl = pd.DataFrame(columns = column)
with tqdm(total=len(datadf)) as pbar:
    for i in range(0,len(datadf)):
        tempdf = datadf.iloc[[i]] # 使用雙重中括號將其轉換為 DataFrame
        if datadf['性別分群'][i] == 0:
            #tempdf = datadf.iloc[i]
            girl = pd.concat([girl, tempdf], ignore_index=True)
        elif datadf['性別分群'][i] == 1:
            #tempdf = datadf.iloc[i]
            boy = pd.concat([boy, tempdf], ignore_index=True)
        pbar.update(1)
print(pd.Series.value_counts(boy['性別分群']))
print(pd.Series.value_counts(girl['性別分群']))

 85%|████████████████████████████████████████████████████████████████▌           | 50216/59143 [00:51<00:15, 583.66it/s]

In [ ]:
import math

datadf['Neutrophil-to-Lymphocyte Ratio'] = 0
for i in range(0,59143):
    datadf['Neutrophil-to-Lymphocyte Ratio'][i] = float(datadf['嗜中性球'][i]) / float(datadf['淋巴球'][i])        
datadf = datadf.reset_index(drop=True)
print(datadf)
print(pd.Series.value_counts(datadf['腸黏膜化生']))

In [ ]:
#分性別
import math
boy['Neutrophil-to-Lymphocyte Ratio'] = 0
girl['Neutrophil-to-Lymphocyte Ratio'] = 0

for i in range(0,35918):
    boy['Neutrophil-to-Lymphocyte Ratio'][i] = float(boy['嗜中性球'][i]) / float(boy['淋巴球'][i])        
boy = boy.reset_index(drop=True)

for i in range(0,23225):
    girl['Neutrophil-to-Lymphocyte Ratio'][i] = float(girl['嗜中性球'][i]) / float(girl['淋巴球'][i])        
girl = girl.reset_index(drop=True)

print(boy)
print(girl)

In [ ]:
#分性別跟年齡
import math
boy0['Neutrophil-to-Lymphocyte Ratio'] = 0
boy1['Neutrophil-to-Lymphocyte Ratio'] = 0
boy2['Neutrophil-to-Lymphocyte Ratio'] = 0
girl0['Neutrophil-to-Lymphocyte Ratio'] = 0
girl1['Neutrophil-to-Lymphocyte Ratio'] = 0
girl2['Neutrophil-to-Lymphocyte Ratio'] = 0

for i in range(0,8175):
    boy0['Neutrophil-to-Lymphocyte Ratio'][i] = float(boy0['嗜中性球'][i]) / float(boy0['淋巴球'][i])        
boy0 = boy0.reset_index(drop=True)

for i in range(0,25310):
    boy1['Neutrophil-to-Lymphocyte Ratio'][i] = float(boy1['嗜中性球'][i]) / float(boy1['淋巴球'][i])        
boy1 = boy1.reset_index(drop=True)

for i in range(0,2433):
    boy2['Neutrophil-to-Lymphocyte Ratio'][i] = float(boy2['嗜中性球'][i]) / float(boy2['淋巴球'][i])        
boy2 = boy2.reset_index(drop=True)

for i in range(0,5122):
    girl0['Neutrophil-to-Lymphocyte Ratio'][i] = float(girl0['嗜中性球'][i]) / float(girl0['淋巴球'][i])        
girl0 = girl0.reset_index(drop=True)

for i in range(0,15891):
    girl1['Neutrophil-to-Lymphocyte Ratio'][i] = float(girl1['嗜中性球'][i]) / float(girl1['淋巴球'][i])        
girl1 = girl1.reset_index(drop=True)

for i in range(0,2212):
    girl2['Neutrophil-to-Lymphocyte Ratio'][i] = float(girl2['嗜中性球'][i]) / float(girl2['淋巴球'][i])        
girl2 = girl2.reset_index(drop=True)

print(boy0)

In [ ]:
import math

datadf['AST * Monocyte to Lymphocyte Ratio'] = 0
for i in range(0,59143):
    datadf['AST * Monocyte to Lymphocyte Ratio'][i] = float(datadf['AST'][i]) * float(datadf['Monocyte to Lymphocyte Ratio'][i])        
datadf = datadf.reset_index(drop=True)
print(pd.Series.value_counts(datadf['腸黏膜化生']))

In [ ]:
#分性別
import math
boy['AST * Monocyte to Lymphocyte Ratio'] = 0
girl['AST * Monocyte to Lymphocyte Ratio'] = 0

for i in range(0,35918):
    boy['AST * Monocyte to Lymphocyte Ratio'][i] = float(boy['AST'][i]) * float(boy['Monocyte to Lymphocyte Ratio'][i])       
boy = boy.reset_index(drop=True)

for i in range(0,23225):
    girl['AST * Monocyte to Lymphocyte Ratio'][i] = float(girl['AST'][i]) * float(girl['Monocyte to Lymphocyte Ratio'][i])       
girl = girl.reset_index(drop=True)

#print(boy)
#print(girl)

In [ ]:
import math

boy0['AST * Monocyte to Lymphocyte Ratio'] = 0
boy1['AST * Monocyte to Lymphocyte Ratio'] = 0
boy2['AST * Monocyte to Lymphocyte Ratio'] = 0
girl0['AST * Monocyte to Lymphocyte Ratio'] = 0
girl1['AST * Monocyte to Lymphocyte Ratio'] = 0
girl2['AST * Monocyte to Lymphocyte Ratio'] = 0

for i in range(0,8175):
    boy0['AST * Monocyte to Lymphocyte Ratio'][i] = float(boy0['AST'][i]) * float(boy0['Monocyte to Lymphocyte Ratio'][i])        
boy0 = boy0.reset_index(drop=True)

for i in range(0,25310):
    boy1['AST * Monocyte to Lymphocyte Ratio'][i] = float(boy1['AST'][i]) * float(boy1['Monocyte to Lymphocyte Ratio'][i])        
boy1 = boy1.reset_index(drop=True)

for i in range(0,2433):
    boy2['AST * Monocyte to Lymphocyte Ratio'][i] = float(boy2['AST'][i]) * float(boy2['Monocyte to Lymphocyte Ratio'][i])        
boy2 = boy2.reset_index(drop=True)

for i in range(0,5122):
    girl0['AST * Monocyte to Lymphocyte Ratio'][i] = float(girl0['AST'][i]) * float(girl0['Monocyte to Lymphocyte Ratio'][i])        
girl0 = girl0.reset_index(drop=True)

for i in range(0,15891):
    girl1['AST * Monocyte to Lymphocyte Ratio'][i] = float(girl1['AST'][i]) * float(girl1['Monocyte to Lymphocyte Ratio'][i])        
girl1 = girl1.reset_index(drop=True)

for i in range(0,2212):
    girl2['AST * Monocyte to Lymphocyte Ratio'][i] = float(girl2['AST'][i]) * float(girl2['Monocyte to Lymphocyte Ratio'][i])        
girl2 = girl2.reset_index(drop=True)

In [ ]:
import math

datadf['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
for i in range(0,59143):
    datadf['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(datadf['腰圍'][i]) * float(datadf['Monocyte to Lymphocyte Ratio'][i])        
datadf = datadf.reset_index(drop=True)

In [ ]:
import math

boy0['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
boy1['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
boy2['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
#boy3['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
#boy4['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
#boy5['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
girl0['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
girl1['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
girl2['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
#girl3['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
#girl4['腰圍 * Monocyte to Lymphocyte Ratio'] = 0
#girl5['腰圍 * Monocyte to Lymphocyte Ratio'] = 0

for i in range(0,8175):
    boy0['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(boy0['腰圍'][i]) * float(boy0['Monocyte to Lymphocyte Ratio'][i])        
boy0 = boy0.reset_index(drop=True)

for i in range(0,25310):
    boy1['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(boy1['腰圍'][i]) * float(boy1['Monocyte to Lymphocyte Ratio'][i])        
boy1 = boy1.reset_index(drop=True)

for i in range(0,2433):
    boy2['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(boy2['腰圍'][i]) * float(boy2['Monocyte to Lymphocyte Ratio'][i])        
boy2 = boy2.reset_index(drop=True)

#for i in range(0,10329):
#    boy3['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(boy3['腰圍'][i]) * float(boy3['Monocyte to Lymphocyte Ratio'][i])        
#boy3 = boy3.reset_index(drop=True)

#for i in range(0,4866):
#    boy4['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(boy4['腰圍'][i]) * float(boy4['Monocyte to Lymphocyte Ratio'][i])        
#boy4 = boy4.reset_index(drop=True)

#for i in range(0,2430):
#    boy5['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(boy5['腰圍'][i]) * float(boy5['Monocyte to Lymphocyte Ratio'][i])        
#boy5 = boy5.reset_index(drop=True)

for i in range(0,5122):
    girl0['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(girl0['腰圍'][i]) * float(girl0['Monocyte to Lymphocyte Ratio'][i])        
girl0 = girl0.reset_index(drop=True)

for i in range(0,15891):
    girl1['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(girl1['腰圍'][i]) * float(girl1['Monocyte to Lymphocyte Ratio'][i])         
girl1 = girl1.reset_index(drop=True)

for i in range(0,2212):
    girl2['腰圍 * Monocyte to Lymphocyte Ratio'][i] = float(girl2['腰圍'][i]) * float(girl2['Monocyte to Lymphocyte Ratio'][i])         
girl2 = girl2.reset_index(drop=True)

In [ ]:
import math

boy['Visceral adiposity index'] = 0
girl['Visceral adiposity index'] = 0

for i in range(0,35918):
    boy['Visceral adiposity index'][i] = (float(boy['腰圍'][i]) / (39.68+(1.88 * float(boy['BMI'][i])))) * (float(boy['三酸甘油脂'][i]) / 1.03) * (1.31 / float(boy['HDL'][i]))       
boy = boy.reset_index(drop=True)

for i in range(0,23225):
    girl['Visceral adiposity index'][i] = (float(girl['腰圍'][i]) / (36.58+(1.89 * float(girl['BMI'][i])))) * (float(girl['三酸甘油脂'][i]) / 0.81) * (1.52 / float(girl['HDL'][i]))     
girl = girl.reset_index(drop=True)

In [ ]:
import math

boy0['Visceral adiposity index'] = 0
boy1['Visceral adiposity index'] = 0
boy2['Visceral adiposity index'] = 0
#boy3['Visceral adiposity index'] = 0
#boy4['Visceral adiposity index'] = 0
#boy5['Visceral adiposity index'] = 0
girl0['Visceral adiposity index'] = 0
girl1['Visceral adiposity index'] = 0
girl2['Visceral adiposity index'] = 0
#girl3['Visceral adiposity index'] = 0
#girl4['Visceral adiposity index'] = 0
#girl5['Visceral adiposity index'] = 0

for i in range(0,8175):
    boy0['Visceral adiposity index'][i] = (float(boy0['腰圍'][i]) / (39.68+(1.88 * float(boy0['BMI'][i])))) * (float(boy0['三酸甘油脂'][i]) / 1.03) * (1.31 / float(boy0['HDL'][i]))       
boy0 = boy0.reset_index(drop=True)

for i in range(0,25310):
    boy1['Visceral adiposity index'][i] = (float(boy1['腰圍'][i]) / (39.68+(1.88 * float(boy1['BMI'][i])))) * (float(boy1['三酸甘油脂'][i]) / 1.03) * (1.31 / float(boy1['HDL'][i]))         
boy1 = boy1.reset_index(drop=True)

for i in range(0,2433):
    boy2['Visceral adiposity index'][i] = (float(boy2['腰圍'][i]) / (39.68+(1.88 * float(boy2['BMI'][i])))) * (float(boy2['三酸甘油脂'][i]) / 1.03) * (1.31 / float(boy2['HDL'][i]))      
boy2 = boy2.reset_index(drop=True)

for i in range(0,5122):
    girl0['Visceral adiposity index'][i] = (float(girl0['腰圍'][i]) / (36.58+(1.89 * float(girl0['BMI'][i])))) * (float(girl0['三酸甘油脂'][i]) / 0.81) * (1.52 / float(girl0['HDL'][i]))  
girl0 = girl0.reset_index(drop=True)

for i in range(0,15891):
    girl1['Visceral adiposity index'][i] = (float(girl1['腰圍'][i]) / (36.58+(1.89 * float(girl1['BMI'][i])))) * (float(girl1['三酸甘油脂'][i]) / 0.81) * (1.52 / float(girl1['HDL'][i]))         
girl1 = girl1.reset_index(drop=True)

for i in range(0,2212):
    girl2['Visceral adiposity index'][i] = (float(girl2['腰圍'][i]) / (36.58+(1.89 * float(girl2['BMI'][i])))) * (float(girl2['三酸甘油脂'][i]) / 0.81) * (1.52 / float(girl2['HDL'][i]))         
girl2 = girl2.reset_index(drop=True)

In [ ]:
bb = boy.copy()
#temp_1
gg = girl.copy()

In [ ]:
temp_1 = boy0.copy()
#temp_1
temp_2 = boy1.copy()
#temp_2
temp_3 = boy2.copy()
#temp_3
temp_4 = girl0.copy()
#temp_4
temp_5 = girl1.copy()
#temp_5
temp_6 = girl2.copy()

In [ ]:
datadf.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
print(datadf)
print(pd.Series.value_counts(datadf['腸黏膜化生']))

In [ ]:
bb.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
gg.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)

In [ ]:
temp_1.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
temp_2.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
temp_3.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
temp_4.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
temp_5.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)
temp_6.drop(['大腸息肉','胃息肉','切片腺瘤','增生性息肉','腺性息肉','大腸切片息肉'], axis=1, inplace=True)

In [ ]:
datadf.replace('',np.nan, inplace=True)
datadf.dropna(axis=0, inplace=True)
datadf.reset_index(drop=True,inplace=True)
print(pd.Series.value_counts(datadf['腸黏膜化生']))
print(datadf)

In [ ]:
bb.replace('',np.nan, inplace=True)
bb.dropna(axis=0, inplace=True)
bb.reset_index(drop=True,inplace=True)
print(pd.Series.value_counts(bb['腸黏膜化生']))
print(bb)

gg.replace('',np.nan, inplace=True)
gg.dropna(axis=0, inplace=True)
gg.reset_index(drop=True,inplace=True)
print(pd.Series.value_counts(gg['腸黏膜化生']))
print(gg)

In [ ]:
temp_1.replace('',np.nan, inplace=True)
temp_1.dropna(axis=0, inplace=True)
temp_1.reset_index(drop=True,inplace=True)
print(pd.Series.value_counts(temp_1['腸黏膜化生']))
#print(temp_1)

temp_2.replace('',np.nan, inplace=True)
temp_2.dropna(axis=0, inplace=True)
temp_2.reset_index(drop=True,inplace=True)
#print(pd.Series.value_counts(temp_2['腸黏膜化生']))
#print(temp_2)

temp_3.replace('',np.nan, inplace=True)
temp_3.dropna(axis=0, inplace=True)
temp_3.reset_index(drop=True,inplace=True)
#print(pd.Series.value_counts(temp_3['腸黏膜化生']))
#print(temp_3)

temp_4.replace('',np.nan, inplace=True)
temp_4.dropna(axis=0, inplace=True)
temp_4.reset_index(drop=True,inplace=True)
#print(pd.Series.value_counts(temp_4['腸黏膜化生']))
#print(temp_4)

temp_5.replace('',np.nan, inplace=True)
temp_5.dropna(axis=0, inplace=True)
temp_5.reset_index(drop=True,inplace=True)
#print(pd.Series.value_counts(temp_5['腸黏膜化生']))
#print(temp_5)

temp_6.replace('',np.nan, inplace=True)
temp_6.dropna(axis=0, inplace=True)
temp_6.reset_index(drop=True,inplace=True)

In [ ]:
print(bb.columns)

In [ ]:
print(temp_1.columns)

2024-08-17 15:17:36,696| ERROR   | Secsh channel 9 open FAILED: No route to host: Connect failed
2024-08-17 15:17:36,698| ERROR   | Could not establish connection from local ('127.0.0.1', 38543) to remote ('140.127.32.23', 27016) side of the tunnel: open new channel ssh error: ChannelException(2, 'Connect failed')
2024-08-17 15:17:49,783| ERROR   | Secsh channel 10 open FAILED: No route to host: Connect failed
2024-08-17 15:17:49,786| ERROR   | Could not establish connection from local ('127.0.0.1', 38543) to remote ('140.127.32.23', 27016) side of the tunnel: open new channel ssh error: ChannelException(2, 'Connect failed')
2024-08-17 15:18:02,871| ERROR   | Secsh channel 11 open FAILED: No route to host: Connect failed
2024-08-17 15:18:02,874| ERROR   | Could not establish connection from local ('127.0.0.1', 38543) to remote ('140.127.32.23', 27016) side of the tunnel: open new channel ssh error: ChannelException(2, 'Connect failed')
2024-08-17 15:18:15,960| ERROR   | Secsh channel 1

In [33]:
import numpy as np
import pandas as pd
import string, sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.preprocessing import label_binarize
#classifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

#resampling
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import xgboost
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.metrics import accuracy_score
def show_report2(trueclass, predicted,classifier,modelss):
    targetName = ['Y','N']
    print('[Model]:',modelss,' ',classifier,':classification report :' )
    print(metrics.classification_report(trueclass, predicted))
    print("Accuracy:",metrics.accuracy_score(trueclass, predicted))
    print("Precision:",metrics.precision_score(trueclass, predicted))
    print("recall:",metrics.recall_score(trueclass,predicted))
    print("f1:",metrics.f1_score(trueclass,predicted))

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [328]:
from imblearn.under_sampling import RandomUnderSampler
#發炎
X = temp_6[['Monocyte to Lymphocyte Ratio', 'SIRI', 'SII', 'Hs-CRP', 'Neutrophil-to-Lymphocyte Ratio', 'HP', '血小板', '淋巴球', '嗜中性球', '單核球']]
y = temp_6['腸黏膜化生']
print(pd.Series.value_counts(y))
X = X.astype(float)

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X, y)
print(pd.Series.value_counts(y_smote))
X_smote = X_smote.reset_index(drop=True)
y_smote = y_smote.reset_index(drop=True)
X_train, X_test,y_train, y_test = train_test_split(X_smote, y_smote ,random_state=42, test_size=0.2, shuffle=True)

腸黏膜化生
0.0    185
1.0     37
Name: count, dtype: int64
腸黏膜化生
0.0    185
1.0    185
Name: count, dtype: int64


In [172]:
from imblearn.under_sampling import RandomUnderSampler
#代謝
X = temp_6[['尿酸', '紅血球色素', '飯前血糖', '飯後血糖', '膽固醇總量', 'BMI', '腰圍', '三酸甘油脂', '收縮壓', '舒張壓', 
        'HDL', '低密度脂蛋白', '超低密度脂蛋白', 'Atherogenic Index of Plasma', 'Triglyceride Glucose Index',
        '脂肪肝', 'Visceral adiposity index']]
y = temp_6['腸黏膜化生']
print(pd.Series.value_counts(y))
X = X.astype(float)

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X, y)
print(pd.Series.value_counts(y_smote))
X_smote = X_smote.reset_index(drop=True)
y_smote = y_smote.reset_index(drop=True)
X_train, X_test,y_train, y_test = train_test_split(X_smote, y_smote ,random_state=42, test_size=0.3, shuffle=True)

腸黏膜化生
0.0    185
1.0     37
Name: count, dtype: int64
腸黏膜化生
0.0    185
1.0    185
Name: count, dtype: int64


In [72]:
#print(datadf)

In [36]:
from imblearn.under_sampling import RandomUnderSampler
#未分群
X = datadf[['性別', '年齡', '尿酸', '紅血球色素', '血小板', '淋巴球', '嗜中性球', '單核球', '飯前血糖', '飯後血糖',
        '膽固醇總量', 'BMI', '腰圍', '三酸甘油脂', 'HDL', '收縮壓', '舒張壓', '低密度脂蛋白', '超低密度脂蛋白',
        'Atherogenic Index of Plasma', 'Triglyceride Glucose Index', 'Monocyte to Lymphocyte Ratio', 'SIRI',
        'SII', 'HP', '牙結石', '齲齒', '牙齒缺失', '慢性齒齦炎', '膽囊息肉', '膽囊結石', '脂肪肝', 'AST', 'Hs-CRP',
        'Neutrophil-to-Lymphocyte Ratio']]
y = datadf['腸黏膜化生']
print(pd.Series.value_counts(y))
X = X.astype(float)

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X, y)
print(pd.Series.value_counts(y_smote))
X_smote = X_smote.reset_index(drop=True)
y_smote = y_smote.reset_index(drop=True)
X_train, X_test,y_train, y_test = train_test_split(X_smote, y_smote ,random_state=42, test_size=0.3, shuffle=True)

腸黏膜化生
0.0    8292
1.0    2088
Name: count, dtype: int64
腸黏膜化生
0.0    8292
1.0    8292
Name: count, dtype: int64


In [81]:
#其他組
X = temp_6[['牙結石', '齲齒', '牙齒缺失', '慢性齒齦炎', '膽囊息肉', '膽囊結石', 'AST']]
y = temp_6['腸黏膜化生']
print(pd.Series.value_counts(y))
#_c = X.columns
# X = resultdf[['性別','年齡','血小板','淋巴球','嗜中性球','膽固醇總量','BMI','腰圍','三酸甘油脂','低密度脂蛋白','Atherogenic Index of Plasma','超低密度脂蛋白','HP','大腸切片息肉']]
# X_1 = resultdf[['性別','年齡','血小板','淋巴球','嗜中性球','膽固醇總量','BMI','腰圍','三酸甘油脂','低密度脂蛋白','Atherogenic Index of Plasma','超低密度脂蛋白','HP','大腸切片息肉']]
X = X.astype(float)
smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X, y)
print(pd.Series.value_counts(y_smote))
X_smote = X_smote.reset_index(drop=True)
y_smote = y_smote.reset_index(drop=True)
X_train, X_test,y_train, y_test = train_test_split(X_smote, y_smote ,random_state=42, test_size=0.3, shuffle=True)

腸黏膜化生
0.0    185
1.0     37
Name: count, dtype: int64
腸黏膜化生
0.0    185
1.0    185
Name: count, dtype: int64


In [164]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# 随机森林超参数调优
rf_param_grid = {
    'n_estimators': [100, 200, 300, 400]
}

rf_grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy', random_state=9527), rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best Random Forest parameters found: ", rf_grid_search.best_params_)
print("Best Random Forest accuracy found: ", rf_grid_search.best_score_)

KeyboardInterrupt: 

In [38]:
# XGBoost超參數調整
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.7, 0.8, 0.9],
}

xgb_clf1 = XGBClassifier(eval_metric='error@0.9')
xgb_grid_search = GridSearchCV(xgb_clf1, xgb_param_grid, cv=10, scoring='recall')
xgb_grid_search.fit(X_train, y_train)

# scoring='accuracy' eval_metric='error@0.9'
# 输出最佳参数
print("Best XGBoost parameters found: ", xgb_grid_search.best_params_)
print("Best XGBoost accuracy found: ", xgb_grid_search.best_score_)

Best XGBoost parameters found:  {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.9}
Best XGBoost accuracy found:  0.8125


In [37]:
def GridSearch(clf,param,tr,ts,clfname):

    grid_search = GridSearchCV(estimator=clf, param_grid=param, n_jobs= -1, verbose=0)
    grid_result = grid_search.fit(tr, ts)

    print(clfname,f"最佳準確率: {grid_result.best_score_}，最佳參數組合：{grid_result.best_params_}")
    return grid_result.best_params_

In [38]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import operator
import scipy
import matplotlib.pyplot as plt
from xgboost import plot_importance

In [159]:
#'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 400, 'subsample': 0.9
# xbg_param = {
#     'learning_rate': np.arange(0.05,0.4,0.05),
#     'n_estimators': np.array([100,200,300,400]),
#     'subsample': np.array([0.3,0.5,0.7,0.9]),
#     'max_depth': np.arange(10,20),
# }
#best_param = GridSearch(xgb_clf1,xbg_param,X_train,y_train,'XGBClassifier')
#xgb1 = XGBClassifier(**best_param)

In [68]:
xgb1 = xgb.XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 100, subsample = 0.6)

In [39]:
def sens_spec(y_true,y_pred):
    tn,fp,fn,tp = confusion_matrix(y_true,y_pred).ravel()

    ##type 1 error
    #fp/(fp+tn)
    error_one = tp/(tp+fn)

    ##type 2 error
    #fn/(fn+tp)
    error_two = tn/(fp+tn)

    # print('XGB')
    # print('SENS:' + str(error_one))
    # print('SPEC:' + str(error_two))
    return error_one,error_two

In [40]:
X = scipy.sparse.csc_matrix(X.astype(float))
trainuse, evaliatiinuse, traininglabel, evaluationlabel = train_test_split(X,y,test_size=0.2, random_state=42)
print(pd.Series.value_counts(traininglabel))
print(pd.Series.value_counts(evaluationlabel))

traininglabels = []
for i in traininglabel:
    traininglabels.append(i)
evaluationlabels = []
for i in evaluationlabel:
    evaluationlabels.append(i)
print(pd.Series.value_counts(traininglabels))
print(pd.Series.value_counts(evaluationlabels))

腸黏膜化生
0.0    6627
1.0    1677
Name: count, dtype: int64
腸黏膜化生
0.0    1665
1.0     411
Name: count, dtype: int64
0.0    6627
1.0    1677
Name: count, dtype: int64
0.0    1665
1.0     411
Name: count, dtype: int64


In [41]:
temp = pd.DataFrame(traininglabels)
trainy = temp[0]
trainy = trainy.astype(int)

print(pd.Series.value_counts(trainy))

0
0    6627
1    1677
Name: count, dtype: int64


In [42]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
#from sklearn import cross_validation, metrics
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import operator
import scipy
import matplotlib.pyplot as plt
from xgboost import plot_importance

def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

In [43]:
from imblearn.under_sampling import RandomUnderSampler
#未分群
X_1 = datadf[['性別', '年齡', '尿酸', '紅血球色素', '血小板', '淋巴球', '嗜中性球', '單核球', '飯前血糖', '飯後血糖',
        '膽固醇總量', 'BMI', '腰圍', '三酸甘油脂', 'HDL', '收縮壓', '舒張壓', '低密度脂蛋白', '超低密度脂蛋白',
        'Atherogenic Index of Plasma', 'Triglyceride Glucose Index', 'Monocyte to Lymphocyte Ratio', 'SIRI',
        'SII', 'HP', '牙結石', '齲齒', '牙齒缺失', '慢性齒齦炎', '膽囊息肉', '膽囊結石', '脂肪肝', 'AST', 'Hs-CRP',
        'Neutrophil-to-Lymphocyte Ratio']]
#y = gg['腸黏膜化生']
#print(pd.Series.value_counts(y))
#X = X.astype(float)


In [71]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from imblearn.over_sampling import BorderlineSMOTE
# scaler = preprocessing.MinMaxScaler()
# X_scaled = scaler.fit_transform(X)
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 5)
#model = KNeighborsClassifier(n_neighbors=4)
score = []
precisionarray = []
recallarray = []
f1scorearray = []
specuse = []
sensuse = []

# 特征重要性存储
important_df = pd.DataFrame(columns = ['Feature','Important'])
important_df['Feature'] = X_1.columns
#xgb_param = {
#    'learning_rate': [0.01, 0.05, 0.1, 0.3],
#    'n_estimators': [100, 200, 300, 400],
#    'max_depth': [3, 5, 7, 9],
#    'subsample': [0.6, 0.7, 0.8, 0.9]
#}
rf_clf_t = RandomForestClassifier(n_estimators = 400, criterion = 'entropy')
#xgb1 = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 9, n_estimators = 100, subsample = 0.7)

RF_score=[]
RF_precision = []
RF_recall = []
RF_f1 = []
RF_sens = []
RF_spec = []
xgb1 = xgb.XGBClassifier(learning_rate = 0.05, max_depth = 10, n_estimators = 200, subsample = 0.7, eval_metric='error@0.7', scale_pos_weight = 5)

countuse = 0
for train_index, test_index in skf.split(trainuse,trainy):
    countuse += 1
    X_train_fold, X_test_fold = trainuse[train_index], trainuse[test_index]
    y_train_fold, y_test_fold = trainy[train_index], trainy[test_index]
    print('before smote train比例\n',pd.Series.value_counts(y_train_fold))
    print('before smote test比例\n',pd.Series.value_counts(y_test_fold))
    smote = SMOTE()
    
    X_train_fold, y_train_fold = smote.fit_resample(X_train_fold, y_train_fold)
    X_test_fold, y_test_fold = smote.fit_resample(X_test_fold, y_test_fold)
    print('after train比例\n',pd.Series.value_counts(y_train_fold))
    print('after test比例\n',pd.Series.value_counts(y_test_fold))
    
    #best_param = GridSearch(xgb_clf1, xgb_param, X_train_fold, y_train_fold, 'XGBClassifier')
    #xgb1 = XGBClassifier(**best_param)
    
    xgb1.fit(X_train_fold, y_train_fold)
    y_pred = xgb1.predict(X_test_fold)
    
    rf_clf_t.fit(X_train_fold, y_train_fold)
    y_pred_rf = rf_clf_t.predict(X_test_fold)
    
    RF_score.append(accuracy_score(y_test_fold,y_pred_rf))
    RF_precision.append(precision_score(y_test_fold,y_pred_rf))
    RF_recall.append(recall_score(y_test_fold,y_pred_rf))
    RF_f1.append(f1_score(y_test_fold,y_pred_rf))
    one,two = sens_spec(y_test_fold,y_pred_rf)
    RF_sens.append(one)
    RF_spec.append(two)
    
    one,two = sens_spec(y_test_fold,y_pred)
    sensuse.append(one)
    specuse.append(two)
    
    
    score.append(accuracy_score(y_test_fold,y_pred))
    precisionarray.append(precision_score(y_test_fold,y_pred))
    recallarray.append(recall_score(y_test_fold,y_pred))
    f1scorearray.append(f1_score(y_test_fold,y_pred))
    show_report2(y_test_fold,y_pred,'XG','FeatureSelection')
    
    feature_importance = xgb1.feature_importances_

    # 获取特征名字，这里使用原始 DataFrame 的列名
    original_columns = list(X_1.columns)
    feature_importance_df = pd.DataFrame({
        'Feature': original_columns,
        'Importance': feature_importance
    })
    
    if countuse == 1:
        for i in range(0,len(feature_importance_df)):
            feature = feature_importance_df['Feature'][i]
            tempindex = important_df.index[important_df['Feature'] == feature].tolist()
            important_df['Important'][tempindex] = float(feature_importance_df['Importance'][i])
    else:
        for i in range(0,len(feature_importance_df)):
            feature = feature_importance_df['Feature'][i]
            tempindex = important_df.index[important_df['Feature'] == feature].tolist()
            important_df['Important'][tempindex] = float(feature_importance_df['Importance'][i]) + float(important_df['Important'][tempindex])

    print('-'*40)
# print('XGB 10 flod score:',score)
print('P+B')
print('-'*40)
print('Max accuracy:',max(score))
print('Mix accuracy:',min(score))
print('AVG accuracy:',np.mean(score))
print('-'*40)
print('Max precision:',max(precisionarray))
print('Mix precision:',min(precisionarray))
print('AVG precision:',np.mean(precisionarray))
print('-'*40)
print('Max recall:',max(recallarray))
print('Mix recall:',min(recallarray))
print('AVG recall:',np.mean(recallarray))
print('-'*40)
print('Max f1_score:',max(f1scorearray))
print('Mix f1_score:',min(f1scorearray))
print('AVG f1_score:',np.mean(f1scorearray))
print('-'*40)
print('Max sens:',max(sensuse))
print('Mix sens:',min(sensuse))
print('AVG sens:',np.mean(sensuse))
print('-'*40)
print('Max spec:',max(specuse))
print('Mix spec:',min(specuse))
print('AVG spec:',np.mean(specuse))

print('RF')
print('-'*40)
print('Max accuracy:',max(RF_score))
print('Mix accuracy:',min(RF_score))
print('AVG accuracy:',np.mean(RF_score))
print('-'*40)
print('Max precision:',max(RF_precision))
print('Mix precision:',min(RF_precision))
print('AVG precision:',np.mean(RF_precision))
print('-'*40)
print('Max recall:',max(RF_recall))
print('Mix recall:',min(RF_recall))
print('AVG recall:',np.mean(RF_recall))
print('-'*40)
print('Max f1_score:',max(RF_f1))
print('Mix f1_score:',min(RF_f1))
print('AVG f1_score:',np.mean(RF_f1))
print('-'*40)
print('AVG RF sens:',np.mean(RF_sens))

for i in range(0,len(important_df)):
    important_df['Important'][i] = important_df['Important'][i] / 10
important_df = important_df.sort_values(by='Important', ascending=False)

print('Feature Importance Ranking:')
important_df

before smote train比例
 0
0    5964
1    1509
Name: count, dtype: int64
before smote test比例
 0
0    663
1    168
Name: count, dtype: int64
after train比例
 0
1    5964
0    5964
Name: count, dtype: int64
after test比例
 0
0    663
1    663
Name: count, dtype: int64
[Model]: FeatureSelection   XG :classification report :
              precision    recall  f1-score   support

           0       0.81      0.78      0.79       663
           1       0.78      0.82      0.80       663

    accuracy                           0.80      1326
   macro avg       0.80      0.80      0.80      1326
weighted avg       0.80      0.80      0.80      1326

Accuracy: 0.7956259426847662
Precision: 0.7840579710144927
recall: 0.8159879336349924
f1: 0.7997043606799703
----------------------------------------
before smote train比例
 0
0    5964
1    1509
Name: count, dtype: int64
before smote test比例
 0
0    663
1    168
Name: count, dtype: int64
after train比例
 0
1    5964
0    5964
Name: count, dtype: int64
after t

,Feature,Important
25,牙結石,0.188657
26,齲齒,0.158005
24,HP,0.108557
0,性別,0.072464
28,慢性齒齦炎,0.065986
29,膽囊息肉,0.053123
31,脂肪肝,0.030713
27,牙齒缺失,0.023953
30,膽囊結石,0.013903
11,BMI,0.013363


In [72]:
import numpy as np
import scipy.stats as st
#from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
confidence_level = 0.95

def compute_confidence_interval(data):
    mean = np.mean(data)
    ci_lower, ci_upper = st.t.interval(confidence_level, df=len(data)-1, loc=mean, scale=st.sem(data))
    return mean, max(0, ci_lower), min(1, ci_upper)

# print('RF')
# rf_accuracy_mean, rf_accuracy_lower, rf_accuracy_upper = compute_confidence_interval(RF_score)
# print(f'accuracy: {rf_accuracy_mean} ({rf_accuracy_lower}, {rf_accuracy_upper})')
# rf_precision_mean, rf_precision_lower, rf_precision_upper = compute_confidence_interval(RF_precision)
# print(f'precision: {rf_precision_mean} ({rf_precision_lower}, {rf_precision_upper})')

# rf_f1_mean, rf_f1_lower, rf_f1_upper = compute_confidence_interval(RF_f1)
# print(f'f1: {rf_f1_mean} ({rf_f1_lower}, {rf_f1_upper})')

# rf_sens_mean, rf_sens_lower, rf_sens_upper = compute_confidence_interval(RF_sens)
# print(f'sens: {rf_sens_mean} ({rf_sens_lower}, {rf_sens_upper})')

# rf_spec_mean, rf_spec_lower, rf_spec_upper = compute_confidence_interval(RF_spec)
# print(f'spec: {rf_spec_mean} ({rf_spec_lower}, {rf_spec_upper})')

# print('')
print('XG')

xg_accuracy_mean, xg_accuracy_lower, xg_accuracy_upper = compute_confidence_interval(score)
print(f'accuracy: {xg_accuracy_mean} ({xg_accuracy_lower}, {xg_accuracy_upper})')

xg_precision_mean, xg_precision_lower, xg_precision_upper = compute_confidence_interval(precisionarray)
print(f'precision: {xg_precision_mean} ({xg_precision_lower}, {xg_precision_upper})')

xg_f1_mean, xg_f1_lower, xg_f1_upper = compute_confidence_interval(f1scorearray)
print(f'f1: {xg_f1_mean} ({xg_f1_lower}, {xg_f1_upper})')

xg_sens_mean, xg_sens_lower, xg_sens_upper = compute_confidence_interval(sensuse)
print(f'sens: {xg_sens_mean} ({xg_sens_lower}, {xg_sens_upper})')

xg_spec_mean, xg_spec_lower, xg_spec_upper = compute_confidence_interval(specuse)
print(f'spec: {xg_spec_mean} ({xg_spec_lower}, {xg_spec_upper})')

XG
accuracy: 0.8002129157496138 (0.7935890141337824, 0.8068368173654453)
precision: 0.7923167405076889 (0.785177695433858, 0.7994557855815198)
f1: 0.8028908177652209 (0.7964138483891252, 0.8093677871413165)
sens: 0.8137938419616045 (0.8065678424230915, 0.8210198415001175)
spec: 0.786631989537623 (0.7784792013565499, 0.7947847777186962)


In [73]:
import numpy as np
import scipy.stats as st
confidence_level = 0.95

print('RF')
print('accuracy:',np.mean(RF_score),st.t.interval(confidence_level,df=len(RF_score)-1,loc=np.mean(RF_score),scale=st.sem(RF_score)))
print('precision:',np.mean(RF_precision),st.t.interval(confidence_level,df=len(RF_precision)-1,loc=np.mean(RF_precision),scale=st.sem(RF_precision)))
print('f1:',np.mean(RF_f1),st.t.interval(confidence_level,df=len(RF_f1)-1,loc=np.mean(RF_f1),scale=st.sem(RF_f1)))
print('sens:',np.mean(RF_sens),st.t.interval(confidence_level,df=len(RF_sens)-1,loc=np.mean(RF_sens),scale=st.sem(RF_sens)))
print('spec:',np.mean(RF_spec),st.t.interval(confidence_level,df=len(RF_spec)-1,loc=np.mean(RF_spec),scale=st.sem(RF_spec)))
print('')
print('XG')
print('accuracy:',np.mean(score),st.t.interval(confidence_level,df=len(score)-1,loc=np.mean(score),scale=st.sem(score)))
print('precision:',np.mean(precisionarray),st.t.interval(confidence_level,df=len(precisionarray)-1,loc=np.mean(precisionarray),scale=st.sem(precisionarray)))
print('f1:',np.mean(f1scorearray),st.t.interval(confidence_level,df=len(f1scorearray)-1,loc=np.mean(f1scorearray),scale=st.sem(f1scorearray)))
print('sens:',np.mean(sensuse),st.t.interval(confidence_level,df=len(sensuse)-1,loc=np.mean(sensuse),scale=st.sem(sensuse)))
print('spec:',np.mean(specuse),st.t.interval(confidence_level,df=len(specuse)-1,loc=np.mean(specuse),scale=st.sem(specuse)))


RF
accuracy: 0.8547609283081116 (0.8497967872623925, 0.8597250693538306)
precision: 0.9924254230905444 (0.9894286475954208, 0.9954221985856679)
f1: 0.8311138433188082 (0.8246910292832927, 0.8375366573543237)
sens: 0.7149546827794562 (0.7067922238183603, 0.7231171417405521)
spec: 0.9945671738367668 (0.9924560451134881, 0.9966783025600456)

XG
accuracy: 0.8002129157496138 (0.7935890141337824, 0.8068368173654453)
precision: 0.7923167405076889 (0.785177695433858, 0.7994557855815198)
f1: 0.8028908177652209 (0.7964138483891252, 0.8093677871413165)
sens: 0.8137938419616045 (0.8065678424230915, 0.8210198415001175)
spec: 0.786631989537623 (0.7784792013565499, 0.7947847777186962)


In [311]:
# XGBoost超参数调优
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.7, 0.8, 0.9]
}

xgb_clf1 = xgb.XGBClassifier()
xgb_grid_search = GridSearchCV(xgb_clf1, xgb_param_grid, cv=5, scoring='accuracy')
xgb_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best XGBoost parameters found: ", xgb_grid_search.best_params_)
print("Best XGBoost accuracy found: ", xgb_grid_search.best_score_)

# 使用最佳参数初始化XGBoost模型
best_params_xgb = xgb_grid_search.best_params_
best_xgb_model = xgb.XGBClassifier(random_state=42, **best_params_xgb)

# 使用交叉验证评估XGBoost模型
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_results_xgb = cross_val_score(best_xgb_model, X_train, y_train, cv=cv, scoring='accuracy')
#mean_cv_accuracy_xgb = np.mean(cv_results_xgb)

#print(f"Mean cross-validated accuracy (XGBoost): {mean_cv_accuracy_xgb}")

# 在完整的训练集上训练XGBoost模型
#best_xgb_model.fit(X_train, y_train)

# 评估XGBoost模型在测试集上的性能
#xgb_results = evaluate_model(best_xgb_model, X_test, y_test)

# 打印XGBoost模型结果
#print("\nXGBoost Results:")
#for metric, value in xgb_results.items():
#    print(f"{metric}: {value}")

KeyboardInterrupt: 

In [113]:
#第一組發炎
# XGBoost超参数调优
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.7, 0.8, 0.9]
}
xgb_grid_search = GridSearchCV(xgb.XGBClassifier(random_state=9527), xgb_param_grid, cv=5, scoring='accuracy')
xgb_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best XGBoost parameters found: ", xgb_grid_search.best_params_)
print("Best XGBoost accuracy found: ", xgb_grid_search.best_score_)

# 使用最佳参数训练最终模型
best_xgb_model = xgb_grid_search.best_estimator_
xgb_results = evaluate_model(best_xgb_model, X_test, y_test)

# 打印XGBoost模型结果
print("\nXGBoost Results:")
for metric, value in xgb_results.items():
    print(f"{metric}: {value}")

Best XGBoost parameters found:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 400, 'subsample': 0.9}
Best XGBoost accuracy found:  0.7778020026278828

XGBoost Results:
accuracy: 0.7926287744227354
roc: 0.7922499299965688
precision: 0.8205128205128205
recall: 0.7448522829006267
f1: 0.7808540591271703


In [114]:
# 使用最佳参数初始化随机森林模型
best_params_rf = {'n_estimators': 200}
best_rf_model = RandomForestClassifier(criterion='entropy', random_state=9527, **best_params_rf)

# 使用交叉验证评估随机森林模型
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_results_rf = cross_val_score(best_rf_model, X_train, y_train, cv=cv, scoring='accuracy')
mean_cv_accuracy_rf = np.mean(cv_results_rf)

print(f"Mean cross-validated accuracy (Random Forest): {mean_cv_accuracy_rf}")

# 在完整的训练集上训练随机森林模型
best_rf_model.fit(X_train, y_train)

# 评估随机森林模型在测试集上的性能
rf_results = evaluate_model(best_rf_model, X_test, y_test)

# 打印随机森林模型结果
print("\nRandom Forest Results:")
for metric, value in rf_results.items():
    print(f"{metric}: {value}")

Mean cross-validated accuracy (Random Forest): 0.759712112982075

Random Forest Results:
accuracy: 0.7775310834813499
roc: 0.7773074511257735
precision: 0.7911153119092628
recall: 0.7493285586392122
f1: 0.7696551724137932


In [115]:
#第一組發炎
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# 随机森林超参数调优
rf_param_grid = {
    'n_estimators': [100, 200, 300, 400]
}

rf_grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy', random_state=9527), rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best Random Forest parameters found: ", rf_grid_search.best_params_)
print("Best Random Forest accuracy found: ", rf_grid_search.best_score_)

# 使用最佳参数训练最终模型
best_rf_model = rf_grid_search.best_estimator_
rf_results = evaluate_model(best_rf_model, X_test, y_test)

# 打印随机森林模型结果
print("Random Forest Results:")
for metric, value in rf_results.items():
    print(f"{metric}: {value}")

Best Random Forest parameters found:  {'n_estimators': 200}
Best Random Forest accuracy found:  0.7532394544877894
Random Forest Results:
accuracy: 0.7775310834813499
roc: 0.7773074511257735
precision: 0.7911153119092628
recall: 0.7493285586392122
f1: 0.7696551724137932


In [117]:
#第一組發炎
# XGBoost超参数调优
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.7, 0.8, 0.9]
}
xgb_grid_search = GridSearchCV(xgb.XGBClassifier(random_state=9527), xgb_param_grid, cv=5, scoring='roc_auc')
xgb_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best XGBoost parameters found: ", xgb_grid_search.best_params_)
print("Best XGBoost accuracy found: ", xgb_grid_search.best_score_)

# 使用最佳参数训练最终模型
best_xgb_model = xgb_grid_search.best_estimator_
xgb_results = evaluate_model(best_xgb_model, X_test, y_test)

# 打印XGBoost模型结果
print("\nXGBoost Results:")
for metric, value in xgb_results.items():
    print(f"{metric}: {value}")

Best XGBoost parameters found:  {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 200, 'subsample': 0.9}
Best XGBoost accuracy found:  0.8505610619583243

XGBoost Results:
accuracy: 0.7904085257548845
roc: 0.7900827815222493
precision: 0.8134110787172012
recall: 0.7493285586392122
f1: 0.7800559179869525


In [118]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# 使用最佳参数初始化XGBoost模型
best_params = {'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 200, 'subsample': 0.6}
best_xgb_model = xgb.XGBClassifier(random_state=9527, **best_params)

# 使用交叉验证评估模型
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=9527)
cv_results = cross_val_score(best_xgb_model, X_train, y_train, cv=cv, scoring='roc_auc')
mean_cv_accuracy = np.mean(cv_results)

print(f"Mean cross-validated accuracy: {mean_cv_accuracy}")

# 在完整的训练集上训练模型
best_xgb_model.fit(X_train, y_train)

# 评估模型在测试集上的性能
xgb_results = evaluate_model(best_xgb_model, X_test, y_test)

# 打印XGBoost模型结果
print("\nXGBoost Results:")
for metric, value in xgb_results.items():
    print(f"{metric}: {value}")

Mean cross-validated accuracy: 0.8274203193542895

XGBoost Results:
accuracy: 0.7748667850799289
roc: 0.774820455988547
precision: 0.7752707581227437
recall: 0.7690241718889884
f1: 0.7721348314606742


In [36]:
#第二組發炎
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# 随机森林超参数调优
rf_param_grid = {
    'n_estimators': [100, 200, 300, 400]
}

rf_grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy', random_state=9527), rf_param_grid, cv=5, 
                              scoring='accuracy', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best Random Forest parameters found: ", rf_grid_search.best_params_)
print("Best Random Forest accuracy found: ", rf_grid_search.best_score_)

# 使用最佳参数训练最终模型
best_rf_model = rf_grid_search.best_estimator_
rf_results = evaluate_model(best_rf_model, X_test, y_test)

# 打印随机森林模型结果
print("Random Forest Results:")
for metric, value in rf_results.items():
    print(f"{metric}: {value}")

Best Random Forest parameters found:  {'n_estimators': 200}
Best Random Forest accuracy found:  0.7564699379275973
Random Forest Results:
accuracy: 0.783303730017762
roc: 0.7830840159489506
precision: 0.7969782813975449
recall: 0.7555953446732319
f1: 0.7757352941176471


In [37]:
#第二組發炎
# XGBoost超参数调优
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.7, 0.8, 0.9]
}
xgb_grid_search = GridSearchCV(xgb.XGBClassifier(random_state=9527), xgb_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best XGBoost parameters found: ", xgb_grid_search.best_params_)
print("Best XGBoost accuracy found: ", xgb_grid_search.best_score_)

# 使用最佳参数训练最终模型
best_xgb_model = xgb_grid_search.best_estimator_
xgb_results = evaluate_model(best_xgb_model, X_test, y_test)

# 打印XGBoost模型结果
print("\nXGBoost Results:")
for metric, value in xgb_results.items():
    print(f"{metric}: {value}")

Best XGBoost parameters found:  {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 400, 'subsample': 0.9}
Best XGBoost accuracy found:  0.7720866295138418

XGBoost Results:
accuracy: 0.7939609236234458
roc: 0.7935431201416632
precision: 0.8255234297108675
recall: 0.7412712623097583
f1: 0.7811320754716982


In [41]:
#X_train, X_test,y_train, y_test = train_test_split(X_smote, y_smote ,random_state=42, test_size=0.3, shuffle=True)

In [38]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

rf_model = RandomForestClassifier(criterion='entropy', random_state=9527)
rf_model.fit(X_train, y_train)
rf_results = evaluate_model(rf_model, X_test, y_test)

In [52]:
# XGBoost模型
xgb_model = xgb.XGBClassifier(random_state=9527)
xgb_model.fit(X_train, y_train)
xgb_results = evaluate_model(xgb_model, X_test, y_test)

In [53]:
# 隨機森林
print("Random Forest Results:")
for metric, value in rf_results.items():
    print(f"{metric}: {value}")

Random Forest Results:
accuracy: 0.7449664429530202
roc: 0.7355296687295232
precision: 0.7391304347826086
recall: 0.8292682926829268
f1: 0.7816091954022988


In [54]:
# XGBoost
print("\nXGBoost Results:")
for metric, value in xgb_results.items():
    print(f"{metric}: {value}")


XGBoost Results:
accuracy: 0.7651006711409396
roc: 0.75928285402257
precision: 0.7701149425287356
recall: 0.8170731707317073
f1: 0.7928994082840236


In [72]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)

rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

# 評估模型
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
#print(rfc.score(X_test, y_test))

Accuracy: 0.8931818181818182


In [74]:
from xgboost import XGBClassifier

xgbc = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

xgbc.fit(X_train, y_train)
#print(xgbc.score(X_test, y_test))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [75]:
# 使用測試資料進行預測
y_pred = xgbc.predict(X_test)

# 評估模型
accuracy = accuracy_score(y_test, y_pred)
#report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
#print(f"Classification Report:\n{report}")

Accuracy: 0.8272646536412078


In [34]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test,y_train, y_test = train_test_split(X_smote, y_smote ,random_state=42, test_size=0.3, shuffle=True)
# 建立 XGBRegressor 模型
xgbrModel = xgb.XGBRegressor()
# 使用訓練資料訓練模型
xgbrModel.fit(X_train, y_train)
# 使用訓練資料預測
predicted = xgbrModel.predict(X_test)

rmse = mean_squared_error(y_test, predicted, squared=False)
print(f"RMSE: {rmse}")


RMSE: 0.30184670246381345
